<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer

In [2]:
RANDOM_STATE = 12345

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')
df.columns

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [7]:
print(df.isna().sum())
df.duplicated().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


153

In [8]:
df = df.drop_duplicates()
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [9]:
for column in df:
    print(column)
    print(df[column].unique())

Пол
[1 0]
Возраст
[41. 46. 29. 21. 28. 43. 39. 25. 36. 32. 38. 23. 40. 34. 26. 42. 27. 33.
 47. 30. 19. 31. 22. 20. 24. 18. 37. 48. 45. 44. 52. 49. 35. 56. 65. 55.
 57. 54. 50. 53. 51. 58. 59. 60. 61. 62.]
Зарплата
[49600. 38000. 21000. 41700. 26100. 41000. 39700. 38600. 49700. 51700.
 36600. 29300. 39500. 55000. 43700. 23300. 48900. 33200. 36900. 43500.
 36100. 26600. 48700. 40400. 38400. 34600. 34800. 36800. 42200. 46300.
 30300. 51000. 28100. 64800. 30400. 45300. 38300. 49500. 19400. 40200.
 31700. 69200. 33100. 31600. 34500. 38700. 39600. 42400. 34900. 30500.
 24200. 49900. 14300. 47000. 44800. 43800. 42700. 35400. 57200. 29600.
 37400. 48100. 33700. 61800. 39400. 15600. 52600. 37600. 52500. 32700.
 51600. 60900. 41800. 47400. 26500. 45900. 35700. 34300. 26700. 25700.
 33300. 31100. 31500. 42100. 37300. 42500. 27300. 46800. 33500. 44300.
 41600. 53900. 40100. 44600. 45000. 32000. 38200. 33000. 38500. 51800.
 33800. 46400. 43200. 31800. 50200. 35100. 30700. 45800. 49300. 42800.
 336

Были дупликаты, их удалил

In [10]:
target = df['Страховые выплаты']
features = df.drop('Страховые выплаты', axis= 1)

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении матрицы признаков на случайно сгенерированную матрицу результат модели не изменится

**Обоснование:** У нас есть модель линейной регресии, которая описывается по формуле: $$ a = Xw + w0 $$



Допустим, мы умножим матрицу признаков X на случайно сгенерированную матрицу A:

$$ X' = XA $$ A - это случайно сгенерированная матрица.

Тогда новая модель линейной регрессии будет выглядеть следующим образом:

$$ a' = X'w' + w0' $$ 
где a' - новый вектор зависимой переменной, X' - новый вектор признаков, w' - новый вектор весов, w0' - новая величина сдвига предсказания.

Тогда $$ X'w' = (XA)w' = X(Aw') $$ 
так как умножение матриц ассоциативно.

Таким образом, новая модель линейной регрессии может быть записана как:

$$ a' = X(Aw') + w0' = Xw + w0 = a $$

Это означает, что результат модели линейной регрессии с умножением признаков на случайно сгенерированную матрицу будет таким же, как и без умножения.

Упрощённая запись формулы предсказания linearRegression <br>

$$
a = X_i w
$$,
где $X_i$ - новые строки признаков<br>
Вместо параметра $w$ подставим формулу её вычисления:
$$
a = X_i w = X_i (X^T X)^{-1} X^T y
$$

Далее будем упрощать формулу используя принцип ассоциативности ($A(BC) = (AB)C$) и то, что транспонированное произведение матриц равно произведению транспонированных матриц взятых в обратном порядке ($(AB)^T = B^T A^T$):

$$
a = X_i (X^T X)^{-1} X^T y = X_iX^{-1}(X^T)^{-1}X^T y = X_iX^{-1}y
$$

При умножении признаков на случайную матрицу $P$, в которой число строк равно чилу столбцов $X$ и $P$ - обратима, то есть из матрицы $P$ можно получить матрицу $P^{-1}$, при этом $PP^{-1} = E$, где $E$ - единичная матрица <br>

Обучая нашу на изменённых признаках (созданных умножением на случайно сгенерированную матрицу), то модель найдёт новые коэффициенты $w'$

$$
w' = ((XP)^T(XP))^{-1}(XP)^Ty
$$

Если подставить их в формулу предсказания и упростить выражение, то получится:

$$
a'= X_iPw' = X_iP((XP)^T(XP))^{-1}(XP)^Ty = X_iP(XP)^{-1}((XP)^T)^{-1}(XP)^Ty = X_iX^{-1}y
$$

В заключение, можно сказать, что результат модели не изменится при умножении исходных признаков на случайно сгенерированную матрицу и, что $a$ = $a'$


$$
a' = X^TX^{-1}X^Ty
$$


Наверное пользуясь свойством матрицы $$
AE = EA = A
$$

можно сказать, что $$ w_p = Pw = PwX$$

## Алгоритм преобразования

**Алгоритм**
- Проверка результатов обычной линейной регрессии.
- Преобразование признаков методом StandardScaler.
- Проверка результатов линейной регрессии с преобразованными StandardScaler признаками.
- Составление новой матрицы, которая будет иметь размер исходной матрицы. 
- Вычисление детерминанта новой матрицы для удостоверения, что она обратима.
- Преобразование новых признаков путем перемножения исходной матрицы на новую случайно сгенерированную.
- Проверка результатов.

**Обоснование**

Новая матрица должна быть такого же размера, как и матрица с исходными признаками, иначе не получится умножить. <br>
А вычисление детерминанта сразу покажет нам являются ли новые признаки зависимыми от предыдущих, без этого доказательство будет нелегитимно 

## Проверка алгоритма

Для проверки алгоритма сперва потребуется разделить на выборки train и test

In [11]:
train_features, test_features,  train_target, test_target = train_test_split(
features, target, test_size= 0.25, random_state= RANDOM_STATE)

In [12]:
model = LinearRegression()
model.fit(train_features, train_target)
model_predict = model.predict(test_features)
score = r2_score(test_target, model_predict)
print('Результат без преобразований',score)

Результат без преобразований 0.42307727492147584


In [16]:
df.columns

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')

In [17]:
num_columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']
target_column = ['Страховые выплаты']

In [18]:
num_pipeline = make_pipeline(StandardScaler())
num_pipeline

Pipeline(steps=[('standardscaler', StandardScaler())])

In [19]:
pipeline = make_pipeline(num_pipeline, LinearRegression())
pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler())])),
                ('linearregression', LinearRegression())])

In [20]:
pipeline.fit(train_features, train_target)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler())])),
                ('linearregression', LinearRegression())])

In [21]:
pipeline_predict = pipeline.predict(test_features)
score_pipeline = r2_score(test_target, pipeline_predict)
print('Результат модели Pipeline с standardScaler', score_pipeline)

Результат модели Pipeline с standardScaler 0.4230772749214825


Результаты  почти не отличаются, отличия лишь в конце, но это вроде как проблема Python, а не из-за ошибки эксперимента.

Теперь нужно написать функцию, которая вернет рандомно сгенерированную матрицу

In [22]:
def random_features(features):
    new_features = features
    n = features.shape[1]
    np.random.seed(RANDOM_STATE)
    new_matrix = np.random.randint(1, 10, (n,n))
    determinant = np.linalg.det(new_matrix)
    while determinant == 0:
        new_matrix = np.random.randint(1, 10, (n,n))
        determinant = np.linalg.det(new_matrix)
    new_features = new_features @ new_matrix
    return new_features

In [23]:
display(features.head())
new_features = random_features(features)
display(new_features.head())

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


,0,1,2,3
0,99452.0,396931.0,347287.0,49899.0
1,76279.0,304140.0,266095.0,38329.0
2,42174.0,168087.0,147058.0,21203.0
3,83532.0,333667.0,291948.0,41861.0
4,52371.0,208890.0,182758.0,26301.0


Результаты привычной нам таблицы сильно изменились и функция не выдала ошибку, детерминант не был равен нулю.

In [24]:
train_features, test_features,  train_target, test_target = train_test_split(
new_features, target, test_size= 0.25, random_state= RANDOM_STATE)

In [25]:
pipeline.fit(train_features, train_target)
new_pipeline_predict = pipeline.predict(test_features)
new_score_pipeline = r2_score(test_target, new_pipeline_predict)
print('Результат модели Pipeline с standardScaler для признаков с умножением на случайно сгенерированную матрицу',
      new_score_pipeline)

Результат модели Pipeline с standardScaler для признаков с умножением на случайно сгенерированную матрицу 0.4230772749215038


Для исходной матрицы были точно такие же результаты. Такие же результаты были у модели с standardscaler <br>
И такие же результаты у модели с случайно сгенерированной матрицей 

## Вывод


Метод защиты персональных данных клиентов путем умножения исходных значений признаков на рандомно сгенерированную матрицу позволяет получить такой же результат предикта Линейной регрессии. 

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.